# Housing Sales Price Study Notebook

## Objectives
*   Answer business requirement 1: 
    * The client is interested to understand the patterns from the house attributes, so the client can learn the most relevant variables that are correlated to  SalePrice.
    * Visualize the relevant variables against the SalePrice.

## Inputs

* outputs/datasets/collection/HousePricing.csv

## Outputs

* generate code that answers business requirement 1 and can be used to build the Streamlit App

## Additional

* The study will be performed on the raw dataset.  

---

# Change working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/housing-pp5/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/housing-pp5'

### Import Libraries

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')


---

# Load Data

In [5]:
import pandas as pd
df = pd.read_csv("outputs/datasets/collection/HousePricing.csv")
df

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageFinish,...,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,SalePrice
0,856,854.0,3.0,No,706,GLQ,150,0.0,548,RFn,...,65.0,196.0,61,5,7,856,0.0,2003,2003,208500
1,1262,0.0,3.0,Gd,978,ALQ,284,NaN,460,RFn,...,80.0,0.0,0,8,6,1262,NaN,1976,1976,181500
2,920,866.0,3.0,Mn,486,GLQ,434,0.0,608,RFn,...,68.0,162.0,42,5,7,920,NaN,2001,2002,223500
3,961,NaN,NaN,No,216,ALQ,540,NaN,642,Unf,...,60.0,0.0,35,5,7,756,NaN,1915,1970,140000
4,1145,NaN,4.0,Av,655,GLQ,490,0.0,836,RFn,...,84.0,350.0,84,5,8,1145,NaN,2000,2000,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,953,694.0,3.0,No,0,Unf,953,NaN,460,RFn,...,62.0,0.0,40,5,6,953,0.0,1999,2000,175000
1456,2073,0.0,NaN,No,790,ALQ,589,NaN,500,Unf,...,85.0,119.0,0,6,6,1542,NaN,1978,1988,210000
1457,1188,1152.0,4.0,No,275,GLQ,877,NaN,252,RFn,...,66.0,0.0,60,9,7,1152,NaN,1941,2006,266500
1458,1078,0.0,2.0,Mn,49,NaN,0,112.0,240,Unf,...,68.0,0.0,0,6,5,1078,NaN,1950,1996,142125


# Data Exploration

We are interested to get more familiar with the dataset, check variable type and distribution, missing levels and what these variables mean in a business context

In [6]:
from ydata_profiling import ProfileReport
pandas_report = ProfileReport(df=df, minimal=True)
pandas_report.to_notebook_iframe()

ModuleNotFoundError: No module named 'ydata_profiling'

# Correlation Study

Correlation analysis function

In [ ]:
def Correlation(df, method, key=False, ascending=True):
    correlation = df.corr(method=method)['SalePrice'].sort_values(key=key, ascending=ascending)[1:]
    
    return correlation

First we will have to replace `NaN` Values to be able to use the One hot Encoder

In [ ]:
ohe = df.fillna('Missing', inplace=False)

In [ ]:
from feature_engine.encoding import OneHotEncoder
encoder = OneHotEncoder(variables=ohe.columns[ohe.dtypes=='object'].to_list(), drop_last=False)
df_ohe = encoder.fit_transform(ohe)
df_ohe.head(3)

In [ ]:
spearman = Correlation(df_ohe, 'spearman', key=abs, ascending=False)
plt.figure(figsize=(8,6))
sns.barplot(x=spearman[:20].values, y=spearman[:20].index)
plt.title('Spearman & Correlation with Sale Price')
plt.xlabel('Correlation Coefficient')
plt.ylabel('Features')
plt.tight_layout()
plt.show()


In [ ]:
pearson = Correlation(df_ohe, 'pearson', key=abs,ascending=False)
plt.figure(figsize=(8, 6))
sns.barplot(x=pearson[:20].values, y=pearson[:20].index)
plt.title('Pearson Correlation with Sale Price')
plt.xlabel('Correlation Coefficient')
plt.ylabel('Features')
plt.tight_layout()
plt.show()

* We can se higher negative correlations that we will further explore 

We will consider the top positive and negative correlation levels at `df_ohe` and will study the associated variables at `df`

Therefore we are studying at df the following variables. We will investigate:

Positive Correlation:
* The Sale price with 1stFlrSF 
* The Sales Price with OverallQual
* The Sales Price with GrLivArea
* The Sales Price with GarageArea
* The Sales Price with TotalBsmtSF
* The Sales Price with YearBuilt 

Negative Correlation:
* The Sale price with KitchenQual_TA
* The Sales Price with GarageFinish_Unf
* The Sales Price with MasVnrArea_0.0
* The Sales Price with GarageYrBlt_Missing
* The Sales Price with GarageFinish_None

We will not furthur study 'YearRemodAdd' since we won't be able to determine if it's same as construction date if no remodeling or additions

In [ ]:
vars_to_study = [     
    '1stFlrSF', 'GarageArea', 
    'GrLivArea', 'OverallQual', 
    'TotalBsmtSF', 'YearBuilt', 
]

# EDA on selected variables

In [ ]:
df_eda = df_ohe.filter(vars_to_study + ['SalePrice'])

## Variables Distribution by SalePrice

- We plot the distribution by SalePrice  

In [ ]:
plt.figure(figsize=(15,10))

for i, attribute in enumerate(vars_to_study, 1):
    plt.subplot(2,3, i)
    sns.scatterplot(data=df_eda, x=df_eda[attribute], y=df['SalePrice'], hue='SalePrice')
    plt.title(f'Sale Price vs. {attribute}')
    plt.xlabel(attribute)
    plt.ylabel('Sale Price')

plt.tight_layout()
plt.show()

Now with the negative correlated variables

In [ ]:
vars_to_study = ['KitchenQual_TA', 'GarageFinish_Unf',
'MasVnrArea_0.0','GarageYrBlt_Missing',
'GarageFinish_None']

In [ ]:
df_eda = df_ohe.filter(vars_to_study + ['SalePrice'])

In [ ]:
plt.figure(figsize=(20,15))
for i, attribute in enumerate(vars_to_study, 1):
    plt.subplot(3,3, i)
    sns.barplot(data=df_eda, x=df_eda[attribute], y='SalePrice')
    plt.xticks(rotation=90)
    plt.title(f'Sale Price by {attribute}')
    plt.xlabel(attribute)
    plt.ylabel('Sale Price')


plt.show()

---

# Conclusions and Next steps

**The correlations and plots interpretation converge.** 

**Top Positive correlated features**

- Houses with larger garages (GarageArea) are likely to have a higher Sale Price, indicating that buyers value spacious garages.

- An increase in total basement square footage (TotalBsmtSF) often leads to an increase in the Sale Price, which indicates that basement area is an important factor in house valuation.

- The Sale Price tends to rise with the size of the first floor (1stFlrSF), which shows the significance of main-level living space in the housing market.

- The Sale Price tends to be higher for houses with better Overall Quality (OverallQual), affirming that quality is a crucial determinant of property value.

- An increase in above-grade living area (GrLivArea) leads to a rise in the Sale Price, which reflects the market's valuation of living space.

- The sale price tends to increase the more up to date the year that they were built (YearBuilt). 

**Top Negative correlated features**

- 'KitchenQual_TA' indicates that the Sale Price of houses with average kitchen quality tends to decrease.

- The Sale Price is typically lower when the garage is not finished, as shown by 'GarageFinish_Unf'.

- Houses without any masonry veneer area tend to have a lower Sale Price, indicated by 'MasVnrArea_0.0'.

- Houses missing the records of the year the garage was built tend to have a decrease on the Sale Price, as shown by 'GarageYrBlt_Missing'.

- The Sale Price usually decreases on houses without garages,  based on 'GarageFinish_None'.

**Buyers are willing to pay premiums for more space and higher quality in homes, as these patterns demonstrate the significance of size and quality in valuation.**

---